In [1]:
import numpy as np
import matplotlib.pyplot as plt
import progressbar

%matplotlib inline

In [2]:
def word_score(jumble, central_letter, word, print_words=False):
    """Check the score according to Spelling Bee rules
    
    jumble : the anagramable letters
    central_letter : the mandatory letter
    word : the anagram made from the jumble letters that determines the score
    
    Returns 0 if invalid word
    """
    if (central_letter in word and set(word) <= set(jumble) and len(word) >= 4):
        score = len(word)
        if len(set(word)) == 7:
            score += 7
        if print_words:
            print(word)
        return score
    else:
        return 0

In [3]:
def total_score(jumble, central_letter, word_list, print_words=False):
    """Check the total score according to Spelling bee rules
    
    Essentially a wrapper for word_score
    jumble : the anagramable letters
    central_letter : the mandatory letter
    word_list : the list of all words to check against
    
    returns the total score and the number of scoring words"""
    score = 0
    count = 0
    for word in word_list:
        temp_score = word_score(jumble, central_letter, word, print_words)
        score += temp_score
        if temp_score > 0:
            count += 1
    return score, count

In [4]:
word_list = np.loadtxt('word_list.txt', dtype=str)
len_all = len(word_list)

In [5]:
# Culling word list of invalid words to save time (>= 4 letters, no "s")
word_list = np.array([word for word in word_list if 's' not in word and len(word) >= 4])
len_culled = len(word_list)

Just out of curiosity, what fraction of words have an "s" in them?

In [6]:
print(f'{(len_all - len_culled) / len_all * 100:.2f} % have the letter "S"')

60.66 % have the letter "S"


Since the rules say no letter can be repeated AND there must be at least one pangram, then the possible 7-letter combinations must be the subset of all words that have 7+ letters, keeping the 7 unique ones from each (with no repeats).

In [7]:
candidates = np.array([''.join(sorted(set(word))) for word in word_list if len(word) >= 7 and len(set(word)) == 7])

In [8]:
candidates = np.unique(candidates)

How many candidates are left?

In [9]:
print(f'{len(candidates)} candidate letter combinations left')

7986 candidate letter combinations left


We will have to test each candidate's total score among all words, as well as for all 7 letters as a central

In [10]:
current_jumble = 'none'
current_total = 0
current_central = 'a'
bar = progressbar.ProgressBar(maxval=len(candidates)).start()
for i, candidate in enumerate(candidates):
    for j in range(7):
        temp_total, temp_count = total_score(candidate, candidate[j], word_list)
        if temp_total > current_total:
            current_jumble = candidate
            current_total = temp_total
            current_central = candidate[j]
    bar.update(i+1)
bar.finish()

100% |########################################################################|


In [11]:
print(f'The winning 7-letter combination is {current_jumble},\n' \
 f'with central letter {current_central}, giving a total score of {current_total}\n' \
 f'from {total_score(current_jumble, current_central, word_list)[1]} different words.')

The winning 7-letter combination is aeginrt,
with central letter r, giving a total score of 4036
from 537 different words.
